## Загрузка датасета

In [1]:
import pandas as pd
import numpy as np

In [2]:
FILE_TYPE = 'csv'
df_original = []
if FILE_TYPE == 'xlsx':
    HABR_DATASET_PATH = 'habr.xlsx'
    df_original = pd.read_excel(HABR_DATASET_PATH)
elif FILE_TYPE == 'csv':
    HABR_DATASET_PATH = 'vacancies_data.csv'
    df_original = pd.read_csv(HABR_DATASET_PATH, encoding='utf8')

In [3]:
df_original

,Ссылка на объявление,Дата размещения,Название,Зарплата от,Зарплата до,Валюта,Вакансия,Стаж,Требования,Город,Тип занятости,Можно удаленно,Описание,Работодатель,Ссылка на работодателя,Рейтинг,Офисы,Число сотрудников,Дата сбора данных
0,https://career.habr.com/vacancies/1000144781,2024-06-27 16:50:48,Архитектор серверной инфраструктуры,NaN,500000.0,₽,Архитектор программного обеспечения,Старший (Senior),JavaScript / Node.js / Python / Elasticsearch ...,Москва,Полный рабочий день,NaN,~~~ О компании и команде\nО нас: Амбициозная к...,Ваша кадровая служба,https://vks.agency,NaN,"Россия, Москва, ул. Атарбекова, д. 4",0.0,2024-07-01 23:06:03
1,https://career.habr.com/vacancies/1000139937,2024-06-28 10:40:49,Руководитель группы FI/CO/FM,NaN,NaN,NaN,Менеджер технической поддержки,NaN,SAP / ABAP,Москва,Полный рабочий день,True,~~~ О компании и команде\nМеждународная сеть м...,Fix Price,https://fix-price.ru/,NaN,"Россия, Москва, Щепкина, д.33",2.0,2024-07-01 23:06:03
2,https://career.habr.com/vacancies/1000143735,2024-06-04 16:24:59,Intern Project Manager,5000.0,25000.0,₽,Менеджер проекта,Стажёр (Intern),Управление проектами / Agile / Управление риск...,"Москва, Санкт-Петербург, Казань",Полный рабочий день,True,~~~ О компании и команде\nКомпания FedAG - быс...,FedAG,https://career.habr.com/companies/fedag,4.90,"Россия, Санкт-Петербург, Институтский переулок 5",1.0,2024-07-01 23:06:04
3,https://career.habr.com/vacancies/1000143869,2024-07-01 12:09:30,"Middle+ Backend (Node.js, TypeScript) Developer",210000.0,NaN,₽,Бэкенд разработчик,Средний (Middle),Node.js / TypeScript / MongoDB / Redis,NaN,Полный рабочий день,True,~~~ О компании и команде\nОптимакрос – крупный...,Optimacros,https://optimacros.com,NaN,"Россия, Москва, Дмитровское шоссе, 81",0.0,2024-07-01 23:06:04
4,https://career.habr.com/vacancies/1000144076,2024-06-25 19:33:45,Системный аналитик,NaN,NaN,NaN,Системный аналитик,NaN,SQL / REST,NaN,NaN,True,~~~ О компании и команде\nОбязанности: Наша ко...,IBS,http://www.ibs.ru/,4.38,"Россия, Омск, ул. Маяковского, 74А\nРоссия, Ша...",12.0,2024-07-01 23:06:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3194,https://career.habr.com/vacancies/1000144598,2024-06-26 11:03:21,Ведущий программист 1C,NaN,250000.0,₽,Программист 1С,Старший (Senior),Разработка под 1С,NaN,Полный рабочий день,True,"~~~ О компании и команде\nКрафтье Компания """" ...",Bullevie,https://bullevie.ru/,NaN,"Россия, Санкт-Петербург, гп.Федоровское, ул.Пр...",0.0,2024-07-01 23:23:50
3195,https://career.habr.com/vacancies/1000143341,2024-06-25 17:01:12,Software Architect (NGFW),NaN,NaN,NaN,Архитектор программного обеспечения,NaN,Разработка программного обеспечения / Проектир...,NaN,NaN,True,~~~ О компании и команде\nNext Generation Fire...,Лаборатория Касперского,https://kaspersky.ru/,4.20,"Россия, Москва, Ленинградское ш., д. 39А, стр....",7.0,2024-07-01 23:23:50
3196,https://career.habr.com/vacancies/1000144180,2024-06-28 14:36:00,Data Center Infrastructure Engineer / Инженер ...,NaN,NaN,NaN,Системный администратор,Средний (Middle),Администрирование Linux / Bash / Osi model,Санкт-Петербург,Полный рабочий день,NaN,~~~ О компании и команде\nYADRO — это российск...,YADRO,https://yadro.com,NaN,"Россия, Москва, ул. Рочдельская, д. 15, стр. 1...",5.0,2024-07-01 23:23:50
3197,https://career.habr.com/vacancies/1000141715,2024-07-01 12:23:49,Ведущий инженер OpenStack,NaN,NaN,NaN,Инженер по доступности сервисов,Старший (Senior),Linux / Ansible / Terraform,Москва,Полный рабочий день,NaN,~~~ О компании и команде\nМы привлекаем амбици...,Cloud.ru,https://cloud.ru,4.75,"Россия, Москва, ул. 2-я Звенигородская, д. 28",4.0,2024-07-01 23:23:51


In [4]:
df_original.isna().sum()

Ссылка на объявление         0
Дата размещения              0
Название                     0
Зарплата от               2667
Зарплата до               2818
Валюта                    2543
Вакансия                     0
Стаж                       756
Требования                   0
Город                     1075
Тип занятости              339
Можно удаленно            1194
Описание                     0
Работодатель                94
Ссылка на работодателя      94
Рейтинг                   1217
Офисы                      406
Число сотрудников           94
Дата сбора данных            0
dtype: int64

In [5]:
valide_cols = [
    'Дата размещения',
    'Вакансия',
    'Стаж',
    'Зарплата от',
    'Зарплата до',
    'Город',
    'Дата сбора данных',
    'Число сотрудников',
    'Тип занятости',
    'Можно удаленно',
]
def get_valide_cols(dataset, cols):
    return dataset[cols]

In [6]:
exchange_rate_template = {
    "₽": 1,
    "$": 92.37,
    "€": 99.53,
    "₸": 0.206
}
def apply_exchange_rate(row, template=exchange_rate_template):
    row["Зарплата от"] = row["Зарплата от"] * exchange_rate_template[row["Валюта"]] if pd.notna(row["Зарплата от"]) else np.nan
    row["Зарплата до"] = row["Зарплата до"] * exchange_rate_template[row["Валюта"]] if pd.notna(row["Зарплата до"]) else np.nan
    return row

In [7]:
def get_difference_percentiles(df):
    filtered_df = df.dropna(subset=['Зарплата от', 'Зарплата до']).copy()
    filtered_df['Разница'] = filtered_df['Зарплата до'] - filtered_df['Зарплата от']
    condition = filtered_df['Разница'] > 0.8 * filtered_df['Зарплата до']
    filtered_df.loc[condition, ['Зарплата от', 'Разница']] = np.nan

    filtered_df = filtered_df.dropna(subset=['Зарплата от', 'Зарплата до'])

    filtered_df = filtered_df[filtered_df['Зарплата от'] >= 1000]
    filtered_df = filtered_df[filtered_df['Зарплата до'] >= 10000]

    filtered_df['Коэффициент разницы'] = np.where(filtered_df['Зарплата до'] != 0,
                                                      filtered_df['Разница'] / filtered_df['Зарплата до'],
                                                      np.nan)

    perc25 = filtered_df['Коэффициент разницы'].quantile(0.25)
    perc50 = filtered_df['Коэффициент разницы'].quantile(0.50)
    perc75 = filtered_df['Коэффициент разницы'].quantile(0.75)

    return perc25, perc50, perc75


def fill_na_salary(df, coef):
    result_df = df.copy()
    result_df['Зарплата до'] = result_df['Зарплата до'].fillna(result_df['Зарплата от'] / (1-coef))
    result_df['Зарплата от'] = result_df['Зарплата от'].fillna(result_df['Зарплата до'] * (1-coef))
    return result_df

In [8]:
import re

def separate_cities(df):
  work_df = df.copy()
  df_new = pd.DataFrame()
  for index in range(work_df.shape[0]):
    row = re.sub("[\(\[].*?[\)\]]", "", work_df['Город'].iloc[index])
    cities = [city.lstrip().rstrip() for city in row.split(',')]
    init_row = work_df.iloc[index].copy()
    for city in cities:
      new_row = init_row.copy()
      new_row['Город'] = city
      df_new = pd.concat([df_new, pd.DataFrame([new_row])], ignore_index=True)
  return df_new

def find_cities_around_threshold(df, threshold=75):
  cities = df['Город'].unique()
  ratios = {}
  for city in cities:
    ratios[city] = df['Город'].value_counts()[city]/df.shape[0]

  answer = []
  all_percentage = 0
  sorted_dict = {k: v for k, v in sorted(ratios.items(), key=lambda item: item[1], reverse=True)}
  for label, pct in sorted_dict.items():
      all_percentage += pct*100
      if all_percentage < threshold:
          answer.append(label)
  return answer

def create_new_cities(df, cities):
  df_new = df.copy()
  for index in range(df_new.shape[0]):
    if (df_new.iloc[index]['Город'] not in cities):
      df_new.at[index, 'Город'] = 'Малый город'
  return df_new

In [9]:
def fillna_experience_with_vacancy(df):
    dataset = df.copy()

    mean_wages = dataset.groupby(['Вакансия', 'Стаж']).agg({'Зарплата от': 'mean', 'Зарплата до': 'mean'}).reset_index()
    mean_wages['Средняя зарплата'] = (mean_wages['Зарплата от'] + mean_wages['Зарплата до']) / 2

    for i, row in dataset.iterrows():
        if pd.isna(row['Стаж']):
            min_wage = row['Зарплата от']
            max_wage = row['Зарплата до']
            avg_wage = (min_wage + max_wage) / 2
            vacancy_name = row['Вакансия']

            closest_experience = None
            smallest_diff = float('inf')
            for _, r in mean_wages[mean_wages['Вакансия'] == vacancy_name].iterrows():
                wage_diff = abs(avg_wage - r['Средняя зарплата'])
                if wage_diff < smallest_diff:
                    smallest_diff = wage_diff
                    closest_experience = r['Стаж']

            dataset.loc[i, "Стаж"] = closest_experience

    return dataset


def fillna_experience_with_wages(df):
    dataset = df.copy()

    mean_wages = dataset.groupby(['Стаж']).agg({'Зарплата от': 'mean', 'Зарплата до': 'mean'}).reset_index()
    mean_wages['Средняя зарплата'] = (mean_wages['Зарплата от'] + mean_wages['Зарплата до']) / 2

    for i, row in dataset.iterrows():
        if pd.isna(row['Стаж']):
            min_wage = row['Зарплата от']
            max_wage = row['Зарплата до']
            avg_wage = (min_wage + max_wage) / 2

            closest_experience = None
            smallest_diff = float('inf')
            for _, r in mean_wages.iterrows():
                wage_diff = abs(avg_wage - r['Средняя зарплата'])
                if wage_diff < smallest_diff:
                    smallest_diff = wage_diff
                    closest_experience = r['Стаж']

            dataset.loc[i, "Стаж"] = closest_experience

    return dataset

Удаляем строки, в которых одновременно отсутствует информация о зарплате в обоих столбцах. Заменяем пропуски нулями и делаем столбец категориальным. Заполняем столбец "Город"

In [10]:
def preprocessing_data(df):
   df_new = df.copy()

   missing_data = df_new[df["Зарплата от"].isnull() & df_new["Зарплата до"].isnull()]
   df_new = df_new.drop(missing_data.index)

   df_new['Можно удаленно'].fillna(0, inplace=True)
   df_new['Можно удаленно'] = df_new['Можно удаленно'].astype('int64')
   df_new['Можно удаленно'] = df_new['Можно удаленно'].apply(lambda x: "Да" if x == 1 else "Нет")

   df_new.loc[(df_new['Можно удаленно'] == "Нет") & (df_new['Город'].isnull()), 'Город'] = 'Не указано'
   df_new.loc[(df_new['Можно удаленно'] == "Да") & (df_new['Город'].isnull()), 'Город'] = 'Удаленно'

   df_new['Тип занятости'].fillna('Не указан', inplace=True)

   df_new['Число сотрудников'].fillna(0, inplace=True)
   df_new['Число сотрудников'] = df_new['Число сотрудников'].astype('int64')

   df_new = df_new.apply(apply_exchange_rate, axis=1)
   df_new.drop("Валюта", axis=1, inplace=True)

   perc25, perc50, perc75 = get_difference_percentiles(df_new)
   df_25 = fill_na_salary(df_new, perc25)

   df_exp = fillna_experience_with_wages(df_25)

   df_result = get_valide_cols(df_exp, valide_cols)
   return df_result


def sep_cities_data(df):
    df_new = df.copy()

    df_new = separate_cities(df_new)
    # cities = find_cities_around_threshold(df_new, 85)
    # df_new = create_new_cities(df_new, cities)

    return df_new

In [11]:
df_prep = preprocessing_data(df_original)
df_sep_cities = sep_cities_data(df_prep)

In [12]:
df_prep

,Дата размещения,Вакансия,Стаж,Зарплата от,Зарплата до,Город,Дата сбора данных,Число сотрудников,Тип занятости,Можно удаленно
0,2024-06-27 16:50:48,Архитектор программного обеспечения,Старший (Senior),388888.888889,500000.000000,Москва,2024-07-01 23:06:03,0,Полный рабочий день,Нет
2,2024-06-04 16:24:59,Менеджер проекта,Стажёр (Intern),5000.000000,25000.000000,"Москва, Санкт-Петербург, Казань",2024-07-01 23:06:04,1,Полный рабочий день,Да
3,2024-07-01 12:09:30,Бэкенд разработчик,Средний (Middle),210000.000000,270000.000000,Удаленно,2024-07-01 23:06:04,0,Полный рабочий день,Да
8,2024-06-04 09:23:18,Менеджер по найму,Стажёр (Intern),77777.777778,100000.000000,Удаленно,2024-07-01 23:06:06,0,Полный рабочий день,Да
18,2024-06-06 13:34:21,Менеджер технической поддержки,Средний (Middle),60000.000000,77142.857143,Удаленно,2024-07-01 23:06:10,0,Полный рабочий день,Да
...,...,...,...,...,...,...,...,...,...,...
3153,2024-07-01 13:29:52,Веб-разработчик,Средний (Middle),200000.000000,257142.857143,Москва,2024-07-01 23:23:36,0,Полный рабочий день,Нет
3160,2024-06-03 15:46:09,Фулстек разработчик,Стажёр (Intern),40000.000000,51428.571429,Москва,2024-07-01 23:23:39,1,Неполный рабочий день,Да
3176,2024-06-26 13:17:01,DevOps-инженер,Старший (Senior),466666.666667,600000.000000,Удаленно,2024-07-01 23:23:44,0,Полный рабочий день,Да
3181,2024-06-27 10:06:33,Фулстек разработчик,Старший (Senior),300000.000000,385714.285714,Екатеринбург,2024-07-01 23:23:45,0,Полный рабочий день,Нет


In [13]:
df_sep_cities

,Дата размещения,Вакансия,Стаж,Зарплата от,Зарплата до,Город,Дата сбора данных,Число сотрудников,Тип занятости,Можно удаленно
0,2024-06-27 16:50:48,Архитектор программного обеспечения,Старший (Senior),388888.888889,500000.000000,Москва,2024-07-01 23:06:03,0,Полный рабочий день,Нет
1,2024-06-04 16:24:59,Менеджер проекта,Стажёр (Intern),5000.000000,25000.000000,Москва,2024-07-01 23:06:04,1,Полный рабочий день,Да
2,2024-06-04 16:24:59,Менеджер проекта,Стажёр (Intern),5000.000000,25000.000000,Санкт-Петербург,2024-07-01 23:06:04,1,Полный рабочий день,Да
3,2024-06-04 16:24:59,Менеджер проекта,Стажёр (Intern),5000.000000,25000.000000,Казань,2024-07-01 23:06:04,1,Полный рабочий день,Да
4,2024-07-01 12:09:30,Бэкенд разработчик,Средний (Middle),210000.000000,270000.000000,Удаленно,2024-07-01 23:06:04,0,Полный рабочий день,Да
...,...,...,...,...,...,...,...,...,...,...
761,2024-07-01 13:29:52,Веб-разработчик,Средний (Middle),200000.000000,257142.857143,Москва,2024-07-01 23:23:36,0,Полный рабочий день,Нет
762,2024-06-03 15:46:09,Фулстек разработчик,Стажёр (Intern),40000.000000,51428.571429,Москва,2024-07-01 23:23:39,1,Неполный рабочий день,Да
763,2024-06-26 13:17:01,DevOps-инженер,Старший (Senior),466666.666667,600000.000000,Удаленно,2024-07-01 23:23:44,0,Полный рабочий день,Да
764,2024-06-27 10:06:33,Фулстек разработчик,Старший (Senior),300000.000000,385714.285714,Екатеринбург,2024-07-01 23:23:45,0,Полный рабочий день,Нет


In [14]:
df_sep_cities.isna().sum()

Дата размещения      0
Вакансия             0
Стаж                 0
Зарплата от          0
Зарплата до          0
Город                0
Дата сбора данных    0
Число сотрудников    0
Тип занятости        0
Можно удаленно       0
dtype: int64

In [15]:
df_sep_cities['Город'].unique()

array(['Москва', 'Санкт-Петербург', 'Казань', 'Удаленно', 'Новый Уренгой',
       'Краснодар', 'Омск', 'Самара', 'Екатеринбург', 'Ростов-на-Дону',
       'Тюмень', 'Нижний Новгород', 'Липецк', 'Барнаул', 'Новосибирск',
       'Томск', 'Стерлитамак', 'Токио', 'Набережные Челны', 'Ставрополь',
       'Псков', 'Лимассол', 'Пенза', 'Саратов', 'Уфа', 'Кемерово',
       'Севастополь', 'Челябинск', 'Воронеж', 'Пушкино', 'Рязань',
       'Брянск', 'Хошимин', 'Ханой', 'Дананг', 'Пермь', 'Кострома',
       'Коломна', 'Таллин', 'Хабаровск', 'Калининград', 'Владивосток',
       'Тель-Авив', 'Красноярск', 'Симферополь', 'Астрахань', 'Иркутск',
       'Ханты-Мансийск', 'Сочи', 'Ижевск', 'Тольятти', 'Энгельс',
       'Майкоп', 'Химки', 'Бердск', 'Курган', 'Ярославль', 'Балашиха',
       'Зеленоград', 'Чебоксары', 'Ульяновск', 'Лос-Анджелес', 'Салехард',
       'Минск', 'Дубай', 'Донецк', 'Тула', 'Иваново', 'Комсомольск'],
      dtype=object)

In [16]:
# Эспорт общей и городской базы данных
df_prep.to_excel("habr_general.xlsx")
df_prep.to_excel("habr_multicity.xlsx")